In [1]:
!pip install -q -U bitsandbytes sqlalchemy pandas psycopg2-binary matplotlib opencv-python

In [4]:
!pip install opencv-python

In [1]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from transformers import AutoModelForCausalLM, GenerationConfig
import torch
import json
import re
import os, time
import yaml
from sqlalchemy.engine import create_engine
import sys,platform
import cv2
import numpy as np
import tempfile

ModuleNotFoundError: No module named 'cv2'

In [6]:
cnfig_file="config.yaml"
def get_from_cnfg(key_path,file_path=cnfig_file):
   try:
       with open(file_path, 'r') as file:
           data = yaml.safe_load(file)
           
       keys = key_path.split('.')
       value = data
       for key in keys:
           value = value[key]
       return value
           
   except FileNotFoundError:
       print(f"File {file_path} not found")
   except yaml.YAMLError as e:
       print(f"YAML parsing error: {e}")
   except KeyError:
       print(f"Key path {key_path} not found")
   except Exception as e:
       print(f"Error: {e}")
   return None

In [7]:
os_name=platform.system()
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import userdata
    engine = create_engine(userdata.get('GCP_DB_URL'))
    gem_key=userdata.get('DB_URL')
    oai_key=userdata.get('DB_URL')
    b_key_id=userdata.get('BB_KEY_ID')
    b_key=userdata.get('BB_KEY')
    source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG/files/'
elif os_name == "Darwin":
    cnfig_file="/Users/bineshkumar/Documents/config.yaml"
    DB_URL = get_from_cnfg("cd_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    b_key_id=get_from_cnfg("bb_token_id",cnfig_file)
    b_key=get_from_cnfg("bb_token",cnfig_file)
    source_folder='/Users/bineshkumar/Documents/mimic-cxr-jpg/2.1.0/files/'
elif os_name == "Linux":
    cnfig_file="/home/bsada1/config.yaml"
    DB_URL = get_from_cnfg("gcp_db_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    b_key_id=get_from_cnfg("bb_token_id",cnfig_file)
    b_key=get_from_cnfg("bb_token",cnfig_file)
    source_folder="/hdd2/bsada1/datasets/MIMIC-CXR/mimic-cxr-jpg-2.1.0.physionet.org/files/"

Key path bb_token_id not found
Key path bb_token not found


In [8]:
engine = create_engine(DB_URL)

In [9]:
def fetch_generation_data(engine):
    import pandas as pd
    import re
    from sqlalchemy import text
    from sqlalchemy.dialects.postgresql.base import PGDialect
    def fake_get_server_version_info(self, connection):
        version_str = connection.execute(text("SELECT version()")).scalar()
        match = re.search(r'v(\d+)\.(\d+)\.(\d+)', version_str)
        if match:
            return tuple(map(int, match.groups()))
        return (13, 0, 0)
    PGDialect._get_server_version_info = fake_get_server_version_info
    query = text("""
    SELECT a.id, a.question_id, a.condition as question_type, a.text as question, a.answer as ground_truth, a.image
    FROM mimicxp.mimic_all_qns a
    where
    a.condition = 'original'
    """)
    return pd.read_sql(query, con=engine)



In [10]:
fetch_generation_data(engine)

,id,question_id,question_type,question,ground_truth,image
0,2ef86c0f-55bf4440-5098b3fc-b9435636-38b5b69c,2752,original,"In the chest X-ray image, what does the distri...",The chest X-ray image reveals increased gas wi...,p13/p13352405/s56801982/2ef86c0f-55bf4440-5098...
1,2ef86c0f-55bf4440-5098b3fc-b9435636-38b5b69c,2753,original,Is there any evidence of a pneumothorax on the...,The current chest X-ray image shows no evidenc...,p13/p13352405/s56801982/2ef86c0f-55bf4440-5098...
2,2ef86c0f-55bf4440-5098b3fc-b9435636-38b5b69c,2754,original,How does the left lung appear in the latest ch...,The left lung has a normal appearance in the l...,p13/p13352405/s56801982/2ef86c0f-55bf4440-5098...
3,8940c466-c9e39762-22971350-b783808a-15d5a1bc,2755,original,How does the position of the chest tubes in th...,The chest tubes remain stable in position when...,p13/p13352405/s56801982/8940c466-c9e39762-2297...
4,8940c466-c9e39762-22971350-b783808a-15d5a1bc,2756,original,What is the visual indication of the soft tiss...,There is an increased size of the soft tissue ...,p13/p13352405/s56801982/8940c466-c9e39762-2297...
...,...,...,...,...,...,...
10356,22582d1c-114af91c-83312668-0af5831e-ceacf04b,2747,original,What findings on the image would suggest that ...,The absence of pneumothorax would be indicated...,p13/p13352405/s55680047/22582d1c-114af91c-8331...
10357,22582d1c-114af91c-83312668-0af5831e-ceacf04b,2748,original,How would the cardiac silhouette be evaluated ...,The cardiac silhouette would have a typical si...,p13/p13352405/s55680047/22582d1c-114af91c-8331...
10358,2ef86c0f-55bf4440-5098b3fc-b9435636-38b5b69c,2749,original,What can be inferred about the stability of th...,The chest X-ray image indicates that the posit...,p13/p13352405/s56801982/2ef86c0f-55bf4440-5098...
10359,2ef86c0f-55bf4440-5098b3fc-b9435636-38b5b69c,2750,original,How has the condition of the right hemithorax ...,"The right hemithorax, including the clips, has...",p13/p13352405/s56801982/2ef86c0f-55bf4440-5098...


In [11]:
import gc
def get_gpu_memory_usage():
    """
    Get current GPU memory usage in MB
    Returns: Memory allocated and memory cached
    """
    # Get memory in bytes and convert to MB
    memory_allocated = torch.cuda.memory_allocated() / 1024**2
    memory_cached = torch.cuda.memory_reserved() / 1024**2
    return memory_allocated, memory_cached

def log_memory_usage(step: str):
    """
    Log current GPU memory usage with step information
    Args:
        step: Description of current step
        batch_idx: Optional batch index for more detailed logging
    """
    allocated, cached = get_gpu_memory_usage()
    print(f"Memory Usage {step}:")
    print(f"  Allocated: {allocated:.2f} MB")
    print(f"  Cached: {cached:.2f} MB")
    print("-" * 50)

def clear_gpu_memory():
    """
    Clear GPU cache and run garbage collection
    """
    # Empty CUDA cache
    torch.cuda.empty_cache()
    # Run Python garbage collection
    gc.collect()

In [12]:
model_id = "StanfordAIMI/CheXagent-8b"
device = "cuda"
dtype = torch.float16
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
generation_config = GenerationConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
def generate_chexagent(prompt, image_path, model=model, processor=processor, generation_config=None):
    image = Image.open(image_path)
    rgb_image = image.convert("RGB")
    images = [rgb_image]
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dtype = torch.float16
    
    model = model.to(device=device, dtype=dtype)
    inputs = processor(
        images=images, 
        text=f" USER: <s>{prompt} ASSISTANT: <s>", 
        return_tensors="pt"
    ).to(device=device, dtype=dtype)
    
    if generation_config is None:
        output = model.generate(**inputs, max_new_tokens=400)[0]
    else:
        output = model.generate(**inputs, generation_config=generation_config)[0]
    
    response = processor.tokenizer.decode(output, skip_special_tokens=True)
    clear_gpu_memory()
    
    return clean_output(response)

In [14]:
def clean_output(text):
    # Extract only the assistant's response
    if "ASSISTANT:" in text:
        return text.split("ASSISTANT:")[1].strip()
    return text

In [15]:
image_path = source_folder + "p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg"
generate_chexagent("Describe whats going on?",image_path)

/NAS/bsada1/huggingface/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'There is a small left apical pneumothorax.'

In [16]:
def check_duplicate(engine,uid,question_id,question, question_category, model_name,image_link):
    query = text("""
        SELECT 1 FROM mimicxp.model_responses_r2
        WHERE 
        uid = :uid
        AND question_id = :question_id and 
        question = :question
          AND question_category = :question_category
          AND model_name = :model_name
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {
            "uid": uid,
            "question_id": question_id,
            "question": question,
            "question_category": question_category,
            "model_name": model_name
        }).fetchone()
    return result is not None

In [17]:
def insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    from sqlalchemy import text
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text("""
                INSERT INTO mimicxp.model_responses_r2
                (uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link)
                VALUES (:uid,:question_id,:question, :question_category, :actual_answer, :model_name, :model_answer, :image_link)
            """), {
                "uid": uid,
                "question_id": question_id,
                "question": question,
                "question_category": question_category,
                "actual_answer": actual_answer,
                "model_name": model_name,
                "model_answer": model_answer,
                "image_link": image_link
            })
            trans.commit()  # Commit the transaction
        except Exception as e:
            trans.rollback()
            raise e



In [18]:
def perturb_image(image, technique="gaussian_noise_compression", **kwargs):
    if technique == "gaussian_noise_compression":
        mean = kwargs.get("mean", 0)
        stddev = kwargs.get("stddev", 25)
        quality = kwargs.get("quality", 90)
        img_float = image.astype(np.float32)
        noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
        noisy_img = img_float + noise
        noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
        result, encimg = cv2.imencode('.jpg', noisy_img, encode_param)
        if result:
            decimg = cv2.imdecode(encimg, cv2.IMREAD_COLOR)
            return decimg
        else:
            raise ValueError("Failed to encode image for compression simulation.")
    else:
        raise ValueError("Unknown perturbation technique specified.")

In [ ]:
def compute_ssim(original, perturbed):
    if original.shape != perturbed.shape:
        raise ValueError("Original and perturbed images must have the same shape.")
    h, w = original.shape[:2]
    win_size = 7
    if h < win_size or w < win_size:
        win_size = min(h, w)
        if win_size % 2 == 0:
            win_size -= 1
    if len(original.shape) == 3:
        ssim_value = ssim(original, perturbed, win_size=win_size, channel_axis=-1)
    else:
        ssim_value = ssim(original, perturbed, win_size=win_size)
    return ssim_value

def encode_image_stream(image_path):
    if os.path.exists(image_path):
        with open(image_path, "rb") as img_file:
            import base64
            return base64.b64encode(img_file.read()).decode("utf-8")
    else:
        return None

In [ ]:
from sqlalchemy import text
from IPython.display import clear_output
model_id = "CheXagent-8b"
import time
error_list = set()
import time
for index, row in fetch_generation_data(engine).iterrows():
    clear_output(wait=True)
    uid=row["id"]
    question_id=row["question_id"]
    question_category=row["question_type"]
    question=row["question"]
    actual_answer=row["ground_truth"]
    image_link = source_folder + row["image"] 
    if check_duplicate(engine,uid,str(question_id), question, "visual_perturb", model_id,image_link):
        print(f"Duplicate record found for question: {question}. Skipping generation.")
        continue
    original_image = cv2.imread(image_link)
    if original_image is None:
        print(f"Image not found: {image_link}")
        failed_images.append(image_link)
        save_failed_images(failed_images)
        continue

    # Apply visual perturbation: slight Gaussian noise and JPEG compression with quality 95
    perturbed_image = perturb_image(original_image, technique="gaussian_noise_compression", stddev=1, quality=95)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
    cv2.imwrite(temp_file.name, perturbed_image)
    perturbed_image_path = temp_file.name
    
    try:    
        generated_answer = generate_chexagent( row["question"], perturbed_image_path)

    except:
        error_list.add(str(uid)+ "-" + str(question_id) + "-"+str(question_category) )
        continue 
    
    time.sleep(5)
    print(f"{model_id} : {generated_answer}")
    print(f"GT: {actual_answer}")
    #insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    insert_model_response(engine, uid,question_id,question, "visual_perturb", actual_answer,model_id , generated_answer,image_link)
    print('--------------------------------')


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
